In [ ]:
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

sectors_physics_and_astronomy={
'Pharmaceutical and Biotechnology':'https://www.glassdoor.com/Job/netherlands-neuroscience-jobs-SRCH_IL.0,11_IN178_KO12,24.htm?industryNId=10005', 
"HR and Staffing":'https://www.glassdoor.com/Job/netherlands-neuroscience-jobs-SRCH_IL.0,11_IN178_KO12,24.htm?industryNId=10026', 
"Sales Engineer":'https://www.glassdoor.com/Job/netherlands-sales-engineer-jobs-SRCH_IL.0,11_IN178_KO12,26.htm?context=Jobs&clickSource=searchBox', 
"Education":'https://www.glassdoor.com/Job/netherlands-neuroscience-jobs-SRCH_IL.0,11_IN178_KO12,24.htm?industryNId=10009', 
"Healthcare":"https://www.glassdoor.com/Job/netherlands-neuroscience-jobs-SRCH_IL.0,11_IN178_KO12,24.htm?industryNId=10012",
}

for j in range(5):
    options = Options()
    options.add_argument("user-data-dir=C:\\Users\\Username\\AppData\\Local\\Google\\Chrome\\User Data")
    driver = webdriver.Chrome(options=options)
    site = list(sectors_physics_and_astronomy.values())[j]
    job_desc_Math_teacher={}
    driver.get(site)
    time.sleep(20)
    initialScroll = 0
    finalScroll = 1000
    start = time.time()
    while True:
        driver.execute_script(f"window.scrollTo({initialScroll}, {finalScroll})")
        initialScroll = finalScroll
        finalScroll += 1000
        time.sleep(3)
        end = time.time()
        if round(end - start) > 25:
            break
    try:                                
        for i in range(2, 12):
            job_desc_butt=driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div[2]/section/article/div[1]/ul/li['+str(i)+']')
            job_desc_butt.click()
            time.sleep(7)
            src = driver.page_source
            soup = BeautifulSoup(src, 'lxml')
            job_descript=soup.find('div', {'class':'jobDescriptionContent desc'})
            job_description=job_descript.get_text()
            job_desc_Math_teacher[i]=job_description
    except Exception as e:
        print(e)
    soup = BeautifulSoup(src, 'lxml')
    src = driver.page_source
    jobs=soup.find_all('li',{'class':"react-job-listing css-wp148e eigr9kq3"})

    images=[]
    salaries=[]
    companies=[]
    job_titles=[]
    locations=[]

    for pro in jobs[1:11]:
        try:
            image=pro.find('img')['src']
            images.append(image)
        except Exception as e:
            print(f"images_error--> {e}")
            images.append("nan")
        try:
            salary=pro.find('span', {"data-test":"detailSalary"}).text.strip()
            salaries.append(salary)
        except Exception as e:
            print(f"salaries_error--> {e}")
            salaries.append("nan")
        try:
            company=pro.find('a', {"class":"css-l2wjgv e1n63ojh0 jobLink"}).text.strip()
            companies.append(company)
        except Exception as e:
            print(f"companies_error--> {e}")
            companies.append("nan")
        try:
            job=pro.find('a', {"class":"jobLink css-1rd3saf eigr9kq2"}).text.strip()
            job_titles.append(job)
        except Exception as e:
            print(f"job_titles_error--> {e}")
            job_titles.append("nan")
        try:
            loc=pro.find('span', {"data-test":"emp-location"}).text.strip()
            locations.append(loc)
        except Exception as e:
            print(f"locations_error--> {e}")
            locations.append("nan")


    pros=[]
    cons=[]

    driver.close()
    driver = webdriver.Chrome(options=options)
    site1 = "https://www.glassdoor.com/Search/results.htm?keyword=&locId=178&locT=N&locName=Netherlands"

    for comp in companies:   
        driver.get(site1)
        time.sleep(20)                         
        comp_input=driver.find_element(By.XPATH, '//*[@id="sc.keyword"] ')  
        time.sleep(5)
        comp_input.click()
        comp_input.clear()
        comp_input.send_keys(comp)
        time.sleep(5)
        try:
            search_button=driver.find_element(By.XPATH, '//*[@id="scBar"]/div/button')
            search_button.click()
            time.sleep(5)

            first_comp=driver.find_element(By.XPATH, '//*[@id="Discover"]/div/div/div[1]/div[1]/div[1]/a[1]')
            first_comp.click()
            time.sleep(5)
                                                                #eiCell cell reviews 
            comp_review_button=driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div/div/div/div[1]/article[1]/div[1]/div[2]/div/div[2]/div/div[1]/div/a[1]')
            comp_review_button.click()
            time.sleep(5)
            
            initialScroll = 0                    
            finalScroll = 1000
            start = time.time()
            while True:
                driver.execute_script(f"window.scrollTo({initialScroll}, {finalScroll})")
                initialScroll = finalScroll
                finalScroll += 1000
                time.sleep(3)
                end = time.time()
                if round(end - start) > 20:
                    break
            src = driver.page_source
            soup = BeautifulSoup(src, 'lxml')

            a =soup.find_all('p', {'class':"mt-0 mb-0 pb v2__EIReviewDetailsV2__bodyColor v2__EIReviewDetailsV2__lineHeightLarge v2__EIReviewDetailsV2__isCollapsed"})
            
            list_cons=[]
            list_pros=[]
            for i in a:
                try:
                    list_cons.append(i.find('span', {'data-test':"cons"}).text.strip())
                except Exception as e:
                    print(e)
            for i in a:
                try:
                    list_pros.append(i.find('span', {'data-test':"pros"}).text.strip())
                except Exception as e:
                    print(e)
            if len(list_cons)==0:
                list_cons.append('NG')
            if len(list_pros)==0:
                list_pros.append('NG')

            pros.append(list_pros)
            cons.append(list_cons)
        except Exception as e:
            print(e)
            pros.append("NG")
            cons.append("NG")

    driver.close()

    import pandas as pd
    print("creating df")
    df_teache=pd.DataFrame({
        "job_titles":job_titles,
        "job_description":list(job_desc_Math_teacher.values()),
        "images":images,
        "locations":locations,
        "salaries":salaries,
        "companies":companies,
        "pros":pros,
        "cons":cons
        }
    )
    df_teache.to_csv(f"Neuroscience_{list(sectors_physics_and_astronomy.keys())[j]}_last.csv", sep=',', index=False)